In [ ]:
from dotenv import load_dotenv
load_dotenv()


In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
llm.invoke("Hello, world!")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 11, 'total_tokens': 20, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-40f3c1e1-2fc1-4bb1-9b49-3a5adcdffaad-0', usage_metadata={'input_tokens': 11, 'output_tokens': 9, 'total_tokens': 20, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [3]:
import os
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")
PINECONE_NAMESPACE = os.getenv("PINECONE_NAMESPACE")
PINECONE_INDEX_NAME_2 = os.getenv("PINECONE_INDEX_NAME_2")
PINECONE_NAMESPACE_2 = os.getenv("PINECONE_NAMESPACE_2")

print(PINECONE_API_KEY, PINECONE_INDEX_NAME, PINECONE_NAMESPACE, PINECONE_INDEX_NAME_2, PINECONE_NAMESPACE_2)

pcsk_5TkRik_EzGTnXtpN1c26rjgbrhySAUqgAAHhR9QEmje4vwZkMoEKiADwUKrPjkEAQwfsTs dur-pregnancy dur-pregnancy-ns ai-drug-info ai-drug-info-ns


# 1. 임산부 금기 여부 Retrieval

In [7]:
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore

def dur_pregnacy_similarity(query):
    embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

    dur_vector_db = PineconeVectorStore(
        embedding=embeddings,
        index_name=PINECONE_INDEX_NAME,
        namespace=PINECONE_NAMESPACE,
        pinecone_api_key=PINECONE_API_KEY
    )
    
    results = dur_vector_db.similarity_search(    
    query=query,
    k=5,
    namespace=PINECONE_NAMESPACE)
    
    return {
        "medicine_name":query,
        "dur_context": "\n".join([doc.page_content for doc in results])
    }
    

In [17]:
from langchain_core.runnables import RunnableLambda

dur_sim = RunnableLambda(dur_pregnacy_similarity)

# 2. 약 정보 Retrieval

In [22]:
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore

def drug_info_similarity(query):
    embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

    dur_vector_db = PineconeVectorStore(
        embedding=embeddings,
        index_name=PINECONE_INDEX_NAME_2,
        namespace=PINECONE_NAMESPACE_2,
        pinecone_api_key=PINECONE_API_KEY
    )
    
    results = dur_vector_db.similarity_search(    
    query=query,
    k=5,
    namespace=PINECONE_NAMESPACE_2)
    
    return {
        "medicine_context": "\n".join([doc.page_content for doc in results])
    }

In [16]:
from langchain_core.runnables import RunnableLambda

drug_simi = RunnableLambda(drug_info_similarity)

In [18]:
chain = dur_sim | drug_simi

In [33]:
from langchain_core.prompts import ChatPromptTemplate

def medicine_info(query):
    prompt = ChatPromptTemplate.from_messages([
            ('system', """
            Persona: You are a knowledgeable and experienced pharmacist with a passion to help people. 
            
            Role: As a pharmacist, your role is to provide simple and clear explanations about medicines for general people.
            
            Examples:
            Your explanation should cover the following:
            
            1. A brief explanation of the medicine, its effectiveness, dosage, precautions, side effects, and storage information.
            2. If the medicine is safe for pregnant women, including any contraindications, warning levels, and what they mean.
            
            Use the context provided below to answer the user's query.
            """),
            ('user', """
            Use the following pieces of retrieved context to answer the question. 
            If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
            
            question : {query}            
            """)
        ]) 


In [30]:
retriever = drug_info_similarity.as_retriever(
    search_type='similarity',
    search_kwargs = {'k':3}
)
query = '복'

retrievals = retriever.batch([query])

AttributeError: 'function' object has no attribute 'as_retriever'